## Fantasy DG 2025 Draft Prep
Analysis for 2025 fantasy dg

In [ ]:
%load_ext autoreload
%autoreload 2

In [42]:

import pandas as pd
import numpy as np
import plotly.express as px
import json

from utils.feature_extraction import calculate_fantasy_points, calculate_composite_scores
from utils.analysis_utils import (player_historic_linechart, 
                                  player_scoring_linechart, 
                                  plot_histogram, 
                                  plot_player_histogram,
                                  player_summary,
                                  ratings_composite,
                                  plot_scatterplot)

In [2]:
#df = pd.read_csv('./data/players_crawled_25.csv')

In [3]:
#scoring_curve_25 = pd.read_csv('./data/points_map_2025.csv')

In [4]:
#points_map_25 = {key: value for key, value in zip(scoring_curve_25['Position'], scoring_curve_25['Points'])}
#points_map_25

In [5]:
#Write points map to json dictionary
# with open('./data/points_map_2025.json', 'w') as f:
#     json.dump(points_map_25, f, indent=4)

In [6]:
df = pd.read_csv('./data/players_crawled_25_updated.csv')
df.head()

,Player,pdga_number,career_events,join_date,rating_current,career_wins,career_earnings,world_rank,stats_data,ratings_data,fantasy_points_22,fantasy_points_23,fantasy_points_24,composite_fp,composite_percentile,frac_calvin
0,Gannon Buhr,75412,157,2015,1059.0,44,"411,712.00",NaN,"[{'Place': ['2', '25', '24', '7', '4', '5', '5...","[{""Rating"": [""1024"", ""1074"", ""1052"", ""1093"", ""...",3139.5,2535.5,5745.5,4622.000,100.0,1.334296
1,Calvin Heimburg,45971,284,2010,1044.0,58,"460,327.31",NaN,"[{'Place': ['3', '16', '6', '2', '2', '3', '1'...","[{""Rating"": [""1014"", ""1058"", ""1070"", ""1039"", ""...",3094.0,4594.5,3949.5,4175.250,99.1,1.205326
2,Ricky Wysocki,38008,426,2009,1053.0,131,"826,674.76",NaN,"[{'Place': ['7', '12', '2', '1', '2', '10', '1...","[{""Rating"": [""1044"", ""1044"", ""1043"", ""1035"", ""...",3500.5,2300.0,5005.0,4058.250,98.3,1.171550
3,Isaac Robinson,50670,182,2011,1046.0,36,"281,949.43",NaN,"[{'Place': ['2', '1', '5', '45', '1', '19', '1...","[{""Rating"": [""987"", ""1053"", ""1069"", ""1052"", ""1...",1676.0,3101.5,3239.5,3191.200,97.4,0.921247
4,Anthony Barela,44382,246,2010,1040.0,39,"242,195.33",NaN,"[{'Place': ['4', '21', '2', '20', '24', '7', '...","[{""Rating"": [""1074"", ""1054"", ""1060"", ""1035"", ""...",933.5,1802.5,3026.0,2597.775,96.6,0.749935


In [7]:
#Create a sample dataframe for aider
#df.head(2).to_csv('./data/players_25_crawled_sample.csv', index=False)

In [ ]:
import plotly.express as px
import pandas as pd




In [22]:
plot_histogram(df, column='fantasy_points_24', annotate_top_n={'Player':7}, nbins=35)

In [10]:
#Composite FP
plot_histogram(df, column='composite_fp', annotate_top_n={'Player':7}, nbins=35)

In [13]:
player_historic_linechart(df, 'Gannon Buhr')

In [19]:
player_scoring_linechart(df=df, player_name='Gannon Buhr')

In [26]:
plot_player_histogram(df=df, column='fantasy_points_24', player_name='Calvin Heimburg', nbins=35)

In [30]:
player_summary(df, 'composite_fp', player_name='Calvin Heimburg')

{'value': 4175.25, 'percentile': 99.1, 'pct_of_max': 90.3, 'rank': '2/117'}

In [39]:
ratings_composite(df, 'Gannon Buhr', decay_rate=0.15, ref_date = '2024-12-01')

1048.2

In [40]:
df['composite_rating'] = df['Player'].apply(lambda x: ratings_composite(df, x))

In [41]:
#Composite rating
plot_histogram(df, column='composite_rating', annotate_top_n={'Player':6}, nbins=35)

In [46]:
plot_scatterplot(df, col_x='fantasy_points_24', col_y='composite_rating', player_name='Calvin Heimburg')

In [44]:
df.to_csv('./data/players_crawled_25_updated2.csv')

In [47]:
plot_scatterplot(df, col_x = 'fantasy_points_23', col_y = 'fantasy_points_24', color_col='composite_rating')